In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [24]:
def is_gzip_file(filepath):
    with open(filepath, 'rb') as f:
        return f.read(2) == b'\x1f\x8b'

In [42]:
csv_path = '../../dataset/pvdb/org/device-3.csv'

In [43]:
if is_gzip_file(csv_path):
  # Read the data directly into a pandas DataFrame
  df = pd.read_csv(csv_path, compression='gzip')
else:
  # Read the data without compression
  df = pd.read_csv(csv_path)

row_count = df.shape[0]
print(f"Total rows: {row_count}")
df.head(5)

Total rows: 83214


,datetime,current_power,current_value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,self_consumption
0,2021-11-06 11:15:00,120588.840909,1581.840909,NaN,119007.000000,1581.840909,1581.840909
1,2021-11-06 11:30:00,122628.934066,1345.988636,NaN,121335.032967,1345.988636,1345.988636
2,2021-11-06 11:45:00,121781.460674,1232.436782,NaN,120580.044944,1232.436782,1232.436782
3,2021-11-06 12:00:00,122953.527473,1739.549451,NaN,121213.978022,1739.549451,1739.549451
4,2021-11-06 12:15:00,122483.966292,1580.325843,NaN,120903.640449,1580.325843,1580.325843


In [44]:
df['datetime'] = df['datetime'].astype(str).str.strip("'\" ")
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['datetime'] = df['datetime'].dt.round('15min')
df

,datetime,current_power,current_value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,self_consumption
0,2021-11-06 11:15:00,120588.840909,1581.840909,NaN,119007.000000,1581.840909,1581.840909
1,2021-11-06 11:30:00,122628.934066,1345.988636,NaN,121335.032967,1345.988636,1345.988636
2,2021-11-06 11:45:00,121781.460674,1232.436782,NaN,120580.044944,1232.436782,1232.436782
3,2021-11-06 12:00:00,122953.527473,1739.549451,NaN,121213.978022,1739.549451,1739.549451
4,2021-11-06 12:15:00,122483.966292,1580.325843,NaN,120903.640449,1580.325843,1580.325843
...,...,...,...,...,...,...,...
83209,2025-10-16 09:30:00,57415.769231,6269.314607,NaN,51303.296703,6269.314607,6269.314607
83210,2025-10-16 09:45:00,91207.157303,7373.000000,NaN,83834.157303,7373.000000,7373.000000
83211,2025-10-16 10:00:00,75290.000000,8385.295455,NaN,67221.890110,8385.295455,8385.295455
83212,2025-10-16 10:15:00,86897.988764,7797.637500,NaN,80273.393258,7797.637500,7797.637500


In [45]:
# Sort the DataFrame by 'Start Time'
df = df.sort_values('datetime')
df

,datetime,current_power,current_value_of_consumption,external_energy_supply,grid_feed_in,internal_power_supply,self_consumption
0,2021-11-06 11:15:00,120588.840909,1581.840909,NaN,119007.000000,1581.840909,1581.840909
1,2021-11-06 11:30:00,122628.934066,1345.988636,NaN,121335.032967,1345.988636,1345.988636
2,2021-11-06 11:45:00,121781.460674,1232.436782,NaN,120580.044944,1232.436782,1232.436782
3,2021-11-06 12:00:00,122953.527473,1739.549451,NaN,121213.978022,1739.549451,1739.549451
4,2021-11-06 12:15:00,122483.966292,1580.325843,NaN,120903.640449,1580.325843,1580.325843
...,...,...,...,...,...,...,...
83209,2025-10-16 09:30:00,57415.769231,6269.314607,NaN,51303.296703,6269.314607,6269.314607
83210,2025-10-16 09:45:00,91207.157303,7373.000000,NaN,83834.157303,7373.000000,7373.000000
83211,2025-10-16 10:00:00,75290.000000,8385.295455,NaN,67221.890110,8385.295455,8385.295455
83212,2025-10-16 10:15:00,86897.988764,7797.637500,NaN,80273.393258,7797.637500,7797.637500


In [46]:
# Define the start and end times
start_time = df['datetime'].min()
end_time = df['datetime'].max()

print(f"device: {start_time} - {end_time}")

device: 2021-11-06 11:15:00 - 2025-10-16 10:30:00


In [47]:
df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,83204,83205,83206,83207,83208,83209,83210,83211,83212,83213
datetime,2021-11-06 11:15:00,2021-11-06 11:30:00,2021-11-06 11:45:00,2021-11-06 12:00:00,2021-11-06 12:15:00,2021-11-06 12:30:00,2021-11-06 12:45:00,2021-11-06 13:00:00,2021-11-06 13:15:00,2021-11-06 13:30:00,...,2025-10-16 08:15:00,2025-10-16 08:30:00,2025-10-16 08:45:00,2025-10-16 09:00:00,2025-10-16 09:15:00,2025-10-16 09:30:00,2025-10-16 09:45:00,2025-10-16 10:00:00,2025-10-16 10:15:00,2025-10-16 10:30:00
current_power,120588.840909,122628.934066,121781.460674,122953.527473,122483.966292,123642.901099,121495.235955,119267.208791,116982.988764,115082.087912,...,16385.404494,32392.417582,28017.078652,33725.318681,50590.550562,57415.769231,91207.157303,75290.0,86897.988764,102741.421053
current_value_of_consumption,1581.840909,1345.988636,1232.436782,1739.549451,1580.325843,1399.032967,2281.303371,1580.444444,2426.808989,2307.010989,...,10523.876404,9105.164835,9013.123596,9465.67033,8361.067416,6269.314607,7373.0,8385.295455,7797.6375,6670.684211
external_energy_supply,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
grid_feed_in,119007.0,121335.032967,120580.044944,121213.978022,120903.640449,122243.868132,119213.932584,117713.406593,114556.179775,112775.076923,...,5861.52809,23287.252747,19003.955056,24259.648352,42229.483146,51303.296703,83834.157303,67221.89011,80273.393258,96070.736842
internal_power_supply,1581.840909,1345.988636,1232.436782,1739.549451,1580.325843,1399.032967,2281.303371,1580.444444,2426.808989,2307.010989,...,10523.876404,9105.164835,9013.123596,9465.67033,8361.067416,6269.314607,7373.0,8385.295455,7797.6375,6670.684211
self_consumption,1581.840909,1345.988636,1232.436782,1739.549451,1580.325843,1399.032967,2281.303371,1580.444444,2426.808989,2307.010989,...,10523.876404,9105.164835,9013.123596,9465.67033,8361.067416,6269.314607,7373.0,8385.295455,7797.6375,6670.684211


In [48]:
# Group by datetime and calculate mean for all numeric columns
df_clean = df.pivot_table(
    index='datetime',
    values=[col for col in df.columns if col != 'datetime'],
    aggfunc='mean'
).reset_index()

In [50]:
print(f"Original shape df1: {df.shape}")
print(f"After deduplication df1: {df_clean.shape}")
print("\nFirst few rows:")
print(df_clean.head())

Original shape df1: (83214, 7)
After deduplication df1: (83214, 7)

First few rows:
             datetime  current_power  current_value_of_consumption  \
0 2021-11-06 11:15:00  120588.840909                   1581.840909   
1 2021-11-06 11:30:00  122628.934066                   1345.988636   
2 2021-11-06 11:45:00  121781.460674                   1232.436782   
3 2021-11-06 12:00:00  122953.527473                   1739.549451   
4 2021-11-06 12:15:00  122483.966292                   1580.325843   

   external_energy_supply   grid_feed_in  internal_power_supply  \
0                     NaN  119007.000000            1581.840909   
1                     NaN  121335.032967            1345.988636   
2                     NaN  120580.044944            1232.436782   
3                     NaN  121213.978022            1739.549451   
4                     NaN  120903.640449            1580.325843   

   self_consumption  
0       1581.840909  
1       1345.988636  
2       1232.436782  
3   

In [51]:
complete_time_series = pd.date_range(start=start_time, end=end_time, freq='15min')
count = complete_time_series1.size
print(f"Number of rows: {count}")

Number of rows: 138238


In [52]:
complete_df = pd.DataFrame({'datetime': complete_time_series})
merged_df = pd.merge(complete_df, df, on='datetime', how='left')

In [53]:
merged_df.to_csv("../../dataset/pvdb/org/device-3.csv", index=False)